<a href="https://colab.research.google.com/github/nithinrk11/Feynn-Labs-Machine-Learning-Internship/blob/main/Project%203/diab_reci_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!git clone https://github.com/nithinrk11/Feynn-Labs-Machine-Learning-Internship.git
%cd Feynn-Labs-Machine-Learning-Internship

Cloning into 'Feynn-Labs-Machine-Learning-Internship'...
remote: Enumerating objects: 388, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 388 (delta 147), reused 162 (delta 89), pack-reused 127 (from 1)
Receiving objects: 100% (388/388), 15.94 MiB | 18.89 MiB/s, done.
Resolving deltas: 100% (187/187), done.
/content/Feynn-Labs-Machine-Learning-Internship/Feynn-Labs-Machine-Learning-Internship/Feynn-Labs-Machine-Learning-Internship


##Installing dependencies

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pickle

In [31]:
data = pd.read_csv('/content/Feynn-Labs-Machine-Learning-Internship/Project 3/filtered_datasets/merged_recipe_df.csv')

In [32]:
data.head()

,Unnamed: 0,Recipe Name,Ingredients,Calories,Sugar (g),Total Fat (g),Saturated Fat (g),Cuisine
0,0,Chicken with Curry Leaves from Andhra Pradesh ...,"3 medium chicken breasts, cut into bite-sized ...",2002.334204,51.733198,73.758990,32.176384,Andhra Pradesh
1,1,Punugulu,"2 Cup Idli/Dosa batter, 0.5 Cup Besan flour, 0...",1057.899202,13.100185,3.945037,0.767981,Andhra Pradesh
2,2,Bid goodbye to grey boiled eggs thanks to this...,"1/3 cup (80ml) sunflower oil, plus extra 1/3 c...",1974.410794,50.171840,133.102144,26.887364,Andhra Pradesh
3,3,Andhra Gunpowder,1 cup roasted gram (also known as chana dalia ...,134.521833,4.477519,5.776929,3.226080,Andhra Pradesh
4,4,Ginger Chutney,"1/2 cup peeled, grated ginger, 1 1/4 teaspoons...",75.018000,7.984223,0.785400,0.163840,Andhra Pradesh


In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641 entries, 0 to 640
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         641 non-null    int64  
 1   Recipe Name        641 non-null    object 
 2   Ingredients        641 non-null    object 
 3   Calories           641 non-null    float64
 4   Sugar (g)          641 non-null    float64
 5   Total Fat (g)      641 non-null    float64
 6   Saturated Fat (g)  641 non-null    float64
 7   Cuisine            641 non-null    object 
dtypes: float64(4), int64(1), object(3)
memory usage: 40.2+ KB


* Setting the sugar level criteria for diabetic types
> although this is set in an average estimated values, by referring to various medical guidlines sources, setting an proper prescribed levels by an professional is preferred.

In [34]:
# Define the sugar level criteria
def is_diabetic_friendly(sugar):
    return sugar < 10 #Average threshold for diabetic-friendly

def is_non_diabetic_friendly(sugar):
    return sugar < 20 #Average threshold for non-diabetic-friendly

# Apply criteria to create new columns
data['diabetic_friendly'] = data['Sugar (g)'].apply(is_diabetic_friendly)
data['non_diabetic_friendly'] = data['Sugar (g)'].apply(is_non_diabetic_friendly)


In [35]:
data.head()

,Unnamed: 0,Recipe Name,Ingredients,Calories,Sugar (g),Total Fat (g),Saturated Fat (g),Cuisine,diabetic_friendly,non_diabetic_friendly
0,0,Chicken with Curry Leaves from Andhra Pradesh ...,"3 medium chicken breasts, cut into bite-sized ...",2002.334204,51.733198,73.758990,32.176384,Andhra Pradesh,False,False
1,1,Punugulu,"2 Cup Idli/Dosa batter, 0.5 Cup Besan flour, 0...",1057.899202,13.100185,3.945037,0.767981,Andhra Pradesh,False,True
2,2,Bid goodbye to grey boiled eggs thanks to this...,"1/3 cup (80ml) sunflower oil, plus extra 1/3 c...",1974.410794,50.171840,133.102144,26.887364,Andhra Pradesh,False,False
3,3,Andhra Gunpowder,1 cup roasted gram (also known as chana dalia ...,134.521833,4.477519,5.776929,3.226080,Andhra Pradesh,True,True
4,4,Ginger Chutney,"1/2 cup peeled, grated ginger, 1 1/4 teaspoons...",75.018000,7.984223,0.785400,0.163840,Andhra Pradesh,True,True


In [36]:
data.columns

Index(['Unnamed: 0', 'Recipe Name', 'Ingredients', 'Calories', 'Sugar (g)',
       'Total Fat (g)', 'Saturated Fat (g)', 'Cuisine', 'diabetic_friendly',
       'non_diabetic_friendly'],
      dtype='object')

##Save and download the new dataframe

In [37]:
#save the new dataframe
data.to_csv('diab_merged_recipe_df.csv', index=False)


In [38]:
#download the csv file
from google.colab import files
files.download('diab_merged_recipe_df.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Machine learning

In [39]:
# Features and targets
X = data[['Calories', 'Sugar (g)', 'Total Fat (g)', 'Saturated Fat (g)']]
y = data[['diabetic_friendly', 'non_diabetic_friendly']]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model for each obesity type
models = {}
for column in y.columns:
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train[column])
    models[column] = model

# Evaluate the model
for column, model in models.items():
    y_pred = model.predict(X_test)
    print(f"Classification Report for {column}:\n", classification_report(y_test[column], y_pred))

Classification Report for diabetic_friendly:
               precision    recall  f1-score   support

       False       1.00      1.00      1.00       103
        True       1.00      1.00      1.00        26

    accuracy                           1.00       129
   macro avg       1.00      1.00      1.00       129
weighted avg       1.00      1.00      1.00       129

Classification Report for non_diabetic_friendly:
               precision    recall  f1-score   support

       False       1.00      1.00      1.00        72
        True       1.00      1.00      1.00        57

    accuracy                           1.00       129
   macro avg       1.00      1.00      1.00       129
weighted avg       1.00      1.00      1.00       129



##Save the trained model

In [40]:
# Save the trained models to a pickle file
with open('trained_diab_recep_model.pkl', 'wb') as file:
    pickle.dump(models, file)


In [41]:
from google.colab import files

files.download('trained_diab_recep_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Make predictions based on diabetic type

In [42]:
#predic diabetic_friendly is true to predict food sugar, calorie, fat levels

# Predict for diabetic_friendly = True
diabetic_friendly_true_indices = y_test['diabetic_friendly'] == True
X_test_diabetic_friendly = X_test[diabetic_friendly_true_indices]

# Use the model trained for 'diabetic_friendly' to predict
y_pred_diabetic_friendly = models['diabetic_friendly'].predict(X_test_diabetic_friendly)

# Display the predicted sugar, calorie, and fat levels for diabetic-friendly foods
predicted_diabetic_friendly_data = X_test_diabetic_friendly[y_pred_diabetic_friendly == True]
print("Predicted Sugar, Calorie, and Fat Levels for Diabetic-Friendly Foods:")
predicted_diabetic_friendly_data


Predicted Sugar, Calorie, and Fat Levels for Diabetic-Friendly Foods:


,Calories,Sugar (g),Total Fat (g),Saturated Fat (g)
304,632.466500,8.549730,33.400815,26.082689
558,366.986427,1.081643,34.976420,5.088824
244,2460.501443,8.077860,153.606151,83.318215
260,483.098500,0.799387,44.270700,24.870360
357,454.713771,0.883457,16.682675,1.467846
605,1430.267759,8.598640,72.159462,13.731716
63,856.992240,1.290706,74.929678,6.253816
54,1139.976500,6.888437,102.912392,66.839012
526,1295.190000,5.315800,30.726300,24.827290
326,4228.258307,7.587491,166.635170,47.168657


In [43]:
#predict diabetic_friendly is true to predict food nameand thier cusine, sugar, calorie, fat levels from the dataset

# Predict for diabetic_friendly = True
diabetic_friendly_true_indices = y_test['diabetic_friendly'] == True
X_test_diabetic_friendly = X_test[diabetic_friendly_true_indices]

# Use the model trained for 'diabetic_friendly' to predict
y_pred_diabetic_friendly = models['diabetic_friendly'].predict(X_test_diabetic_friendly)

# Get the indices of the predicted diabetic-friendly foods
predicted_diabetic_friendly_indices = y_test[y_test['diabetic_friendly'] == True].index[y_pred_diabetic_friendly == True]

# Display the predicted food name, cuisine, sugar, calorie, and fat levels for diabetic-friendly foods
predicted_diabetic_friendly_data = data.loc[predicted_diabetic_friendly_indices, ['Recipe Name', 'Cuisine', 'Sugar (g)', 'Calories', 'Total Fat (g)']]
print("Predicted Food Name, Cuisine, Sugar, Calorie, and Fat Levels for Diabetic-Friendly Foods:")
predicted_diabetic_friendly_data


Predicted Food Name, Cuisine, Sugar, Calorie, and Fat Levels for Diabetic-Friendly Foods:


,Recipe Name,Cuisine,Sugar (g),Calories,Total Fat (g)
304,Instant Pot Avial (Kerala Mixed Vegetable Curry),Kerala,8.549730,632.466500,33.400815
558,Salmon In Bengali Mustard Sauce,Bengali,1.081643,366.986427,34.976420
244,Smita Chandra’s Malabar Mussels,Kerala,8.077860,2460.501443,153.606151
260,Ladoo (Kerala Style),Kerala,0.799387,483.098500,44.270700
357,Kothmir Wadi | Kothambir Wadi – Maharashtrian ...,Maharashtrian,0.883457,454.713771,16.682675
605,Machhere Jhol (Bengali Fish Curry),Bengali,8.598640,1430.267759,72.159462
63,Goan Prawns Fry,Goan,1.290706,856.992240,74.929678
54,"Goan Prawns Curry Recipe, How to make Goan",Goan,6.888437,1139.976500,102.912392
526,Idiyappam Recipe - Kerala and Tamil Nadu C,Tamil Nadu,5.315800,1295.190000,30.726300
326,Lucknowi Chicken Biryani,Lucknowi,7.587491,4228.258307,166.635170
